## 문제 6

[Kaggle 형] train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8
...


In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
267387,A,B,B,C,B,B,A,E,G,G,...,0.22443,0.69220,0.30912,0.47134,0.53290,0.97139,0.59414,0.36856,0.42951,8.13741
410037,A,A,A,A,B,B,A,E,C,F,...,0.43278,0.97199,0.62596,0.40193,0.49181,0.56479,0.39386,0.36247,0.71541,7.40427
139373,A,A,B,A,B,B,A,E,C,J,...,0.36238,0.41310,0.27010,0.40112,0.53052,0.35635,0.33341,0.33354,0.82542,7.98022
113765,A,A,A,C,B,B,A,E,C,G,...,0.32670,0.42301,0.28331,0.46425,0.38302,0.38054,0.68827,0.34895,0.73375,7.26085
179915,A,B,A,C,B,B,A,E,E,L,...,0.20121,0.37466,0.46316,0.75578,0.55676,0.31308,0.43157,0.32302,0.18028,7.26448


In [10]:
df_train.select_dtypes('object')

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9
id,,,,,,,,,,
267387,A,B,B,C,B,B,A,E,G,G
410037,A,A,A,A,B,B,A,E,C,F
139373,A,A,B,A,B,B,A,E,C,J
113765,A,A,A,C,B,B,A,E,C,G
179915,A,B,A,C,B,B,A,E,E,L
...,...,...,...,...,...,...,...,...,...,...
320975,A,A,A,C,B,B,A,E,C,I
29591,A,B,A,C,B,B,A,E,C,F
46717,A,B,B,C,B,D,A,E,G,F


In [4]:
df_test.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
464589,A,A,A,A,B,B,A,E,A,H,...,0.28209,0.39187,0.37791,0.35714,0.43700,0.32367,0.54198,0.37136,0.38006,0.73431
39160,A,A,A,C,B,D,A,E,C,F,...,0.27939,0.40697,0.72917,0.68887,0.46054,0.58760,0.91763,0.85978,0.38625,0.38306
55733,A,A,A,A,B,D,A,E,C,F,...,0.27850,0.56523,0.71266,0.88031,0.45310,0.59965,0.80885,0.60043,0.86587,0.71623
97702,A,A,A,C,B,D,A,E,C,K,...,0.61301,0.93478,0.37334,0.94991,0.97579,0.83216,0.74999,0.81728,0.61453,0.81351
337310,A,B,A,C,B,D,A,E,C,H,...,0.72733,0.68813,0.45907,0.69450,0.44414,0.45540,0.41404,0.47312,0.70891,0.35307


In [15]:
from scipy.stats import norm

mu_A, std_A = 6.769, 0.616
mu_B, std_B = 8.123, 0.527

df_train = df_train.assign(
    prob_A = 1 - norm.cdf(df_train['target'], loc=mu_A, scale=std_A),
    prob_B = norm.cdf(df_train['target'], loc=mu_B, scale=std_B),
    targetA = df_train['target'] < 7.45
)
df_train_sub = df_train.query('prob_A < 0.01 or prob_B < 0.01').copy()

In [18]:
# 문제 3에서 썼던 방법 

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

cat_cols =['cat{}'.format(i) for i in range(10)]
cont_cols = ['cont{}'.format(i) for i in range(14)]
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(sparse=False), cat_cols),
    ('pt', 'passthrough', cont_cols)
])

clf_xgb = make_pipeline(ct, xgb.XGBClassifier(
    max_depth=2,
    reg_alpha=0.1,
    reg_lambda=0.1,
    colsample_bytree=0.25,
    n_estimators=500,
    random_state=123))
X_cols_clf = cat_cols + cont_cols
clf_xgb.fit(df_train_sub[X_cols_clf], df_train_sub['targetA'])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                n_values=None,
                                                                sparse=False),
                                                  ['cat0', 'cat1', 'cat2',
                                    

In [21]:
clf_xgb.predict_proba(df_test[X_cols_clf])[:, 1]

array([0.6788843 , 0.4183271 , 0.6291298 , ..., 0.518598  , 0.34293258,
       0.621071  ], dtype=float32)

In [22]:
df_train['prd_A'] = clf_xgb.predict_proba(df_train[X_cols_clf])[:, 1]
df_test['prd_A'] = clf_xgb.predict_proba(df_test[X_cols_clf])[:, 1]

In [23]:
for i in range(14):
    col = 'cont{}'.format(i)
    qt = df_train[col].quantile(np.linspace(0, 1, 101))
    qt.iloc[[0, -1]] = [-np.inf, np.inf]
    q_val = pd.cut(df_train[col], qt, right=True)
    q_mean = df_train.groupby(q_val)['target'].mean()
    df_train[col + '_q'] = q_val.map(q_mean).astype(float)
    df_test[col + '_q'] = pd.cut(df_test[col], qt, right=True).map(q_mean).astype(float)

In [33]:
from sklearn.linear_model import Ridge
q_cols = ['cont{}_q'.format(i) for i in range(14)]
ct_ridge = ColumnTransformer([
    ('ohe', OneHotEncoder(sparse=False, drop='first'), cat_cols),
    ('pt', 'passthrough', q_cols + ['prd_A'])
])
X_cols_lr = cat_cols + q_cols + ['prd_A']
reg_ridge = make_pipeline(ct_ridge, Ridge(alpha=0.1))
reg_ridge.fit(df_train[X_cols_lr], df_train['target'])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop='first',
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                n_values=None,
                                                                sparse=False),
                                                  ['cat0', 'cat1', 'cat2',
                                 

In [34]:
reg_ridge.predict(df_test[X_cols_lr])

array([7.2046082 , 7.68520105, 7.36238266, ..., 7.49836826, 7.74754477,
       7.43696894])

In [30]:
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')

In [35]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df_ans['target'], reg_ridge.predict(df_test[X_cols_lr])) ** 0.5

0.8480306065425177

In [38]:
pd.DataFrame({
    'id': df_test.index,
    'target': reg_ridge.predict(df_test[X_cols_lr])
}).set_index('id').to_csv('answer6.csv')